# Playground

In [1]:
import torch
from uda import UNet, UNetConfig
from uda.datasets import CC359, CC359Config
from uda.utils import patchify, unpatchify
from pathlib import Path
from uda.metrics import EpochDice, SurfaceDice, flatten_output
import numpy as np

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

data_dir = Path("/tmp/data/CC359")

Using device: cuda:0


In [2]:
dataset_conf = CC359Config(
    vendor="GE_3",
    fold=1,
    flatten=True,
)
dataset = CC359(data_dir, dataset_conf, train=False)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

Loading files: 100%|██████████| 20/20 [00:04<00:00,  4.54it/s]


In [3]:
config = UNetConfig(
    out_channels=1,
    dim=2,
    encoder_blocks=((1, 8),),
    decoder_blocks=((8, 8),),
)
model = UNet(config)
model = model.to(device)

In [4]:
with torch.no_grad():
    data, label, pred = [*zip(*[(data, label, model(data.to(device)).cpu()) for data, label in dataloader])]

y = torch.cat(label)
y_pred = torch.cat(pred)

y_pred, y = flatten_output((y_pred, y))

y_pred.shape

torch.Size([251658240])

In [6]:
epoch_dice = EpochDice(orig_shape=dataset.PADDING_SHAPE, reduce_mean=False)
dsc = epoch_dice.compute_fn(y_pred, y)

dsc.shape

torch.Size([20])

In [ ]:
surface_dice = SurfaceDice(dataset.spacing_mm, 0.5, reduce_mean=False, orig_shape=dataset.PADDING_SHAPE, patch_dims=dataset.patch_dims)
sdsc = surface_dice.compute_fn(y_pred, y)

sdsc